# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient()

In [2]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
db = client.get_database("companies_s")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies_s')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [4]:
db.companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies_s'), 'companies')

In [5]:
a = list(db.companies_s.find({"name":"Babelgum"},{"name":1}))

a

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [7]:
# b = db.companies_s.find({"number_of_employees":{"$gt":5000}})

# b = n.limit(20)

# b = n.sort([("number_of_employees",1)])

b = list(db.companies_s.find({"number_of_employees":{"$gt":5000}}).sort([("number_of_employees",1)]).limit(20))

len(b)

20

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [19]:
c = list(db.companies_s.find({"founded_year":{"$gte":2000, "$lte":2005}},{"name":1,"founded_year":1}))

c[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'founded_year': 2005}

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [25]:
d = list(db.companies_s.find({"ipo.valuation_amount":{"$gt":100000000},"founded_year":{"$lt": 2010}},{"name":1,"ipo":1}))

d[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
 'name': 'Facebook',
 'ipo': {'valuation_amount': 104000000000,
  'valuation_currency_code': 'USD',
  'pub_year': 2012,
  'pub_month': 5,
  'pub_day': 18,
  'stock_symbol': 'NASDAQ:FB'}}

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [33]:
e = list(db.companies_s.find({"number_of_employees":{"$lt":1000}, "founded_year":{"$lt":2005}}).sort([("number_of_employees",1)]).limit(10))

len(e)

10

### 6. All the companies that don't include the `partners` field.

In [21]:
e = list(db.companies_s.find({"name":{"$not":"partners"}}))

e

OperationFailure: $not needs a regex or a document, full error: {'ok': 0.0, 'errmsg': '$not needs a regex or a document', 'code': 2, 'codeName': 'BadValue'}

### 7. All the companies that have a null type of value on the `category_code` field.

In [35]:
f = db.companies_s.find({"category_code":null})

f

NameError: name 'null' is not defined

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [36]:
g = list(db.companies_s.find({"number_of_employees":{"$gte":100, "$lt":1000}},{"name":1,"number_of_employees":1}))

g[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'number_of_employees': 600}

### 9. Order all the companies by their IPO price in a descending order.

In [43]:
h = list(db.companies_s.find().sort([("ipo.valuation",-1)]))
         
len(h)

18801

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [46]:
i = list(db.companies_s.find().sort([("number_of_employees",1)]).limit(10))

len(i)

10

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [50]:
filter_q = {"founded_month":{"$gt":6}}
project = {"_id":0, "name":1,"founded_month":1}
cur = db.companies_s.find(filter_q,project)
list(cur.limit(1000))[:20]

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'Google', 'founded_month': 9},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'delicious', 'founded_month': 9},
 {'name': 'AllPeers', 'founded_month': 8}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [60]:
filter_q = {"founded_year":{"$lt":2000}}
project = {"_id":0, "name":1,"acquisition.price_amount":{"$gt":10_000_000}}
cur = db.companies_s.find(filter_q,project)
list(cur.limit(10))[:20]

OperationFailure: Expression $gt takes exactly 2 arguments. 1 were passed in., full error: {'ok': 0.0, 'errmsg': 'Expression $gt takes exactly 2 arguments. 1 were passed in.', 'code': 16020, 'codeName': 'Location16020'}

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
filter_q = {"acquisition.acquired_year":{"$gt"2010}}
project = {"_id":0, "name":1,"founded_month":1}
cur = db.companies_s.find(filter_q,project)
list(cur.limit(1000))[:20]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code